<a href="https://colab.research.google.com/github/mushhub/my-first-blockchain/blob/main/Staking_ETH.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
// SPDX-License-Identifier: MIT
pragma solidity ^0.8.0;

import "https://github.com/OpenZeppelin/openzeppelin-contracts/blob/master/contracts/security/ReentrancyGuard.sol";
import "https://github.com/OpenZeppelin/openzeppelin-contracts/blob/master/contracts/access/Ownable.sol";

/**
 * @title ETHステーキングコントラクト
 * @dev ユーザーがETHをステークして報酬を得るためのコントラクト
 */
contract ETHStaking is ReentrancyGuard, Ownable {
    // ステーキング情報を格納する構造体
    struct StakeInfo {
        uint256 amount;        // ステークした量
        uint256 stakedAt;      // ステークした時間
        uint256 lastClaimAt;   // 最後に報酬を請求した時間
    }

    // ユーザーごとのステーキング情報
    mapping(address => StakeInfo) public stakes;

    // 総ステーキング量
    uint256 public totalStaked;

    // APY（年間利回り）: 5%
    uint256 public annualRate = 5;

    // 最小ステーキング量: 0.01 ETH
    uint256 public minStakeAmount = 0.01 ether;

    // 最低ステーキング期間: 1日
    uint256 public minStakingPeriod = 1 days;

    // イベント
    event Staked(address indexed user, uint256 amount);
    event Unstaked(address indexed user, uint256 amount);
    event RewardClaimed(address indexed user, uint256 amount);

    /**
     * @dev ETHをステークするための関数
     */
    function stake() external payable nonReentrant {
        require(msg.value >= minStakeAmount, "Stake amount too low");

        StakeInfo storage userStake = stakes[msg.sender];

        // すでにステークしている場合は、未請求の報酬を計算
        if (userStake.amount > 0) {
            uint256 reward = calculateReward(msg.sender);
            userStake.lastClaimAt = block.timestamp;

            // 報酬をユーザーのステーキング額に追加
            userStake.amount += reward;
        } else {
            // 初めてステークする場合
            userStake.stakedAt = block.timestamp;
            userStake.lastClaimAt = block.timestamp;
        }

        userStake.amount += msg.value;
        totalStaked += msg.value;

        emit Staked(msg.sender, msg.value);
    }

    /**
     * @dev ステークしたETHと報酬を引き出す関数
     */
    function unstake() external nonReentrant {
        StakeInfo storage userStake = stakes[msg.sender];
        require(userStake.amount > 0, "No staked ETH");
        require(block.timestamp >= userStake.stakedAt + minStakingPeriod, "Staking period not met");

        uint256 reward = calculateReward(msg.sender);
        uint256 totalAmount = userStake.amount + reward;

        // ユーザーのステーキング情報をリセット
        totalStaked -= userStake.amount;
        userStake.amount = 0;
        userStake.stakedAt = 0;
        userStake.lastClaimAt = 0;

        // ETHを送金
        (bool success, ) = payable(msg.sender).call{value: totalAmount}("");
        require(success, "Transfer failed");

        emit Unstaked(msg.sender, totalAmount);
    }

    /**
     * @dev 報酬のみを請求する関数
     */
    function claimReward() external nonReentrant {
        uint256 reward = calculateReward(msg.sender);
        require(reward > 0, "No rewards to claim");

        // 最後の請求時間を更新
        stakes[msg.sender].lastClaimAt = block.timestamp;

        // 報酬を送金
        (bool success, ) = payable(msg.sender).call{value: reward}("");
        require(success, "Transfer failed");

        emit RewardClaimed(msg.sender, reward);
    }

    /**
     * @dev 報酬を計算する内部関数
     * @param user ユーザーのアドレス
     * @return 計算された報酬
     */
    function calculateReward(address user) public view returns (uint256) {
        StakeInfo storage userStake = stakes[user];
        if (userStake.amount == 0) return 0;

        // 前回の請求からの経過時間（秒数）
        uint256 timeElapsed = block.timestamp - userStake.lastClaimAt;

        // 年間利回りを秒単位に変換
        uint256 rewardRate = (annualRate * 1e18) / (365 days);

        // 報酬を計算: 金額 * 経過時間 * 秒間レート
        uint256 reward = (userStake.amount * timeElapsed * rewardRate) / 1e18;

        return reward;
    }

    /**
     * @dev ユーザーのステーキング情報を取得する関数
     * @param user ユーザーのアドレス
     * @return ステークした量、ステーキング開始時間、最後の報酬請求時間
     */
    function getStakeInfo(address user) external view returns (uint256, uint256, uint256) {
        return (stakes[user].amount, stakes[user].stakedAt, stakes[user].lastClaimAt);
    }

    /**
     * @dev 年間利回りを変更する関数（オーナーのみ）
     * @param newRate 新しい年間利回り（%）
     */
    function setAnnualRate(uint256 newRate) external onlyOwner {
        require(newRate <= 100, "Rate too high");
        annualRate = newRate;
    }

    /**
     * @dev 最小ステーキング量を変更する関数（オーナーのみ）
     * @param newAmount 新しい最小ステーキング量
     */
    function setMinStakeAmount(uint256 newAmount) external onlyOwner {
        minStakeAmount = newAmount;
    }

    /**
     * @dev 最低ステーキング期間を変更する関数（オーナーのみ）
     * @param newPeriod 新しい最低ステーキング期間（秒）
     */
    function setMinStakingPeriod(uint256 newPeriod) external onlyOwner {
        minStakingPeriod = newPeriod;
    }

    /**
     * @dev コントラクトがETHを受け取るための関数
     */
    receive() external payable {}
}